# Imports

In [1]:
#imports required to get selenium to work on Colabs
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromiu

In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Selenium
Use selenium to to get page source. Cannot use requests as it is blocked for that method.

In [3]:
# a driver for total and advanced stats
driver_total = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver_adv = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [0]:
# Use selenium to to get source for page 
# cannot use requests as it is blocked for that method
# array of html pages sources from Basketball-Reference
total_source = []
advanced_source = []
# start and end dates for required years
start_year = 2018
end_year = 2020
years = list(range(start_year,end_year+1))
for i in years:
  driver_total.get('https://www.basketball-reference.com/leagues/NBA_%s_totals.html' % (i)) 
  driver_adv.get('https://www.basketball-reference.com/leagues/NBA_%s_advanced.html' % (i))
  total_source.append(driver_total.page_source)
  advanced_source.append(driver_adv.page_source)
driver_total.close()
driver_adv.close()


# Beautiful Soup
Use bs to sort through the HTML as it is a faster interface to that of selenium 

In [0]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [0]:
# returns a pandas DataFrame with all players data from a page source
def nba_parse_data(page_source):
  soup = BeautifulSoup(page_source, 'html.parser')
  print(soup.title)
  table = soup.find_all("table")[0]
  tbody = table.find('tbody')
  tr = tbody.find_all('tr')
  dataset = []
  for player in tr:
    data1 = []
    for col in player.find_all('td'):
      data1.append(col.text)
    dataset.append(data1)
  return pd.DataFrame(data=dataset)

In [7]:
# creates the tables for each NBA Player Total Season Stats table
data_per = [] 
for season in total_source:
  df = nba_parse_data(season)
  df.columns = ['PLAYER',	'POS'	,'AGE',	'TEAM',	'G'	,'GS',	'MP',	'FG'	
                ,'FGA','FG%',	'3P'	,'3PA',	'3P%',	'2P',	'2PA'	,'2P%','eFG%'
                ,	'FT',	'FTA',	'FT%',	'ORB',	'DRB',	'TRB',	'AST',	'STL'
                ,	'BLK',	'TOV'	,'PF'	,'PTS']
  df = df.drop_duplicates()
  df = df.drop('MP',axis=1)
  data_per.append(df)

# creates the tables for each NBA Player Per Game Advanced Stats table
data_adv = [] 
for season in advanced_source:
  df = nba_parse_data(season)
  df.columns = ['PLAYER',	'POS'	,'AGE',	'TEAM',	'G',	'MP',	'PER',	'TS%'
  ,'3PAr','FTr',	'ORB%',	'DRB%',	'TRB%',	'AST%',	'STL%',	'BLK%',	'TOV%',	'USG%'
  ,	'blank1','OWS',	'DWS',	'WS',	'WS/48'	,'blank2','OBPM',	'DBPM',	'BPM',	'VORP']
  df = df.drop_duplicates()
  df = df.drop(['blank1','blank2'],axis=1)
  data_adv.append(df)

<title>2017-18 NBA Player Stats: Totals | Basketball-Reference.com</title>
<title>2018-19 NBA Player Stats: Totals | Basketball-Reference.com</title>
<title>2019-20 NBA Player Stats: Totals | Basketball-Reference.com</title>
<title>2017-18 NBA Player Stats: Advanced | Basketball-Reference.com</title>
<title>2018-19 NBA Player Stats: Advanced | Basketball-Reference.com</title>
<title>2019-20 NBA Player Stats: Advanced | Basketball-Reference.com</title>


In [0]:
# Merges the Per Game and Advanced stats for each year
frames = []
for i in range(len(years)):
  adv = data_adv[i]
  per = data_per[i]
  table = pd.merge(adv,per,on=['PLAYER',	'POS',	'AGE',	'TEAM','G'])
  # insert a YEAR column
  table.insert(loc=1,column='YEAR',value=years[i])
  frames.append(table)

In [0]:
data = pd.concat(frames)
data = data.dropna()

In [0]:
# #save DATA to csv
data.to_csv('season_stats_2018-2020.csv')
# !cp vote_mvp.csv "drive/My Drive/"

In [11]:
data

,PLAYER,YEAR,POS,AGE,TEAM,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,GS,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,2018,SG,24,OKC,75,1134,9.0,.567,.759,.158,2.5,8.9,5.6,3.4,1.7,0.6,7.4,12.7,1.3,1.0,2.2,.094,-1.9,0.4,-1.5,0.1,8,115,291,.395,84,221,.380,31,70,.443,.540,39,46,.848,26,88,114,28,38,8,25,124,353
1,Quincy Acy,2018,PF,27,BRK,70,1359,8.2,.525,.800,.164,3.1,17.1,10.0,6.0,1.2,1.6,13.3,14.4,-0.1,1.1,1.0,.036,-2.6,0.1,-2.5,-0.2,8,130,365,.356,102,292,.349,28,73,.384,.496,49,60,.817,40,217,257,57,33,29,60,149,411
2,Steven Adams,2018,C,24,OKC,76,2487,20.6,.630,.003,.402,16.6,13.9,15.3,5.5,1.8,2.8,13.3,16.7,6.7,3.0,9.7,.187,1.7,-0.6,1.1,2.0,76,448,712,.629,0,2,.000,448,710,.631,.629,160,286,.559,384,301,685,88,92,78,128,215,1056
3,Bam Adebayo,2018,C,20,MIA,69,1368,15.7,.570,.021,.526,9.7,21.6,15.6,11.0,1.2,2.5,13.6,15.9,2.3,1.9,4.2,.148,-1.1,0.7,-0.4,0.6,19,174,340,.512,0,7,.000,174,333,.523,.512,129,179,.721,118,263,381,101,32,41,66,138,477
4,Arron Afflalo,2018,SG,32,ORL,53,682,5.8,.516,.432,.160,0.6,10.1,5.3,6.2,0.3,1.1,10.8,12.5,-0.1,0.2,0.1,.009,-3.8,-1.5,-5.4,-0.6,3,65,162,.401,27,70,.386,38,92,.413,.485,22,26,.846,4,62,66,30,4,9,21,56,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,Thaddeus Young,2020,PF,31,CHI,64,1591,13.3,.521,.369,.120,6.2,16.1,10.9,11.3,2.8,1.4,13.9,19.7,-0.2,2.1,1.9,.058,-1.4,0.2,-1.1,0.4,16,269,601,.448,79,222,.356,190,379,.501,.513,42,72,.583,94,221,315,117,92,23,102,134,659
621,Trae Young,2020,PG,21,ATL,60,2120,23.9,.595,.455,.448,1.6,11.5,6.5,45.6,1.4,0.3,16.2,34.9,5.4,0.5,5.9,.134,6.3,-2.3,4.0,3.2,60,546,1249,.437,205,568,.361,341,681,.501,.519,481,559,.860,32,223,255,560,65,8,289,104,1778
622,Cody Zeller,2020,C,27,CHO,58,1341,18.8,.576,.157,.374,12.6,21.2,16.7,11.3,1.5,1.7,11.9,20.8,2.4,1.2,3.6,.129,0.2,-0.8,-0.6,0.5,39,251,479,.524,18,75,.240,233,404,.577,.543,122,179,.682,160,251,411,88,40,25,75,140,642
623,Ante Žižić,2020,C,23,CLE,22,221,16.4,.597,.000,.264,9.0,24.4,16.6,4.2,1.5,1.9,11.1,17.5,0.3,0.2,0.5,.106,-1.7,-1.5,-3.2,-0.1,0,41,72,.569,0,0,,41,72,.569,.569,14,19,.737,18,48,66,6,7,5,10,27,96
